## Intro

The purpose of today's class is to explore data using **interactive visualizations**. Interactivity is a key part of modern dataviz. It's a way to allow users of your visualizations get their own feel for the data ... to create richer visualization, where people who use your work can expose more of the data by exploring.


## Part 1: Video Lectures and Reading

Starting this week, we'll be playing around with *explanatory data visualization*. Roughly speaking this means using data visualization to communicate your results to others. Thus, there are new things to think about. 

Until today we have worked with static data visualization. However, exploratory data analysis means to be able to explore the multi-faceted nature of data and *interactive dataviz* is a handy tool to do it! It allows to play with the data: Toggle the view. Zoom. Drag. Show more details. All those things. Those are a key part of modern data visualization. 

The video below provides context about these points.

We start with the video and then read a bit from a scientific article about types of explanatory dataviz. (*The video is from an old version of the class that used D3, so just ignore those parts.*).

[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/yHKYMGwefso/0.jpg)](https://www.youtube.com/watch?v=yHKYMGwefso)

> *Exercises*: Explanatory data visualization
> * What are the three key elements to keep in mind when you design an explanatory visualization?
> * In the video I talk about (1) *overview first*,  (2) *zoom and filter*,  (3) *details on demand*. 
>   - Go online and find a visualization that follows these principles (don't use one from the video). 
>   - Explain how your video achieves (1)-(3). It might be useful to use screenshots to illustrate your explanation.
> * Explain in your own words: How is explanatory data analysis different from exploratory data analysis?

We are looking at the first Tableau dashboard in this article about endangered animal species:
https://www.tableau.com/blog/july-viz-roundup-40813

* This interactive visualization starts with an overview that shows a shape of all the different animals listed as endangered, from critical endangered to least concern. It would actually be nice with a more detailed overview showing a few statistics as well.
* It then allows for interactivity where you can zoom and filter, eg. by clicking on each animal and getting more information about which areas they live in and their level of endangerment.
* Details on demand: By hovering over each animal shape, you actually get the same information as by clicking, there is no tooltip with additional information on each animal. Could have been a good touch to give the viewer a deeper level of exploration.

**Explanatory vs exploratory data analysis**
These two types of data analysis vary a lot. You would always start out with an exploratory data anlysis, which is the part where you investigate the quality of the data set and the interesting patterns and stories hidden in the data. The explanatory data analysis is then the next step, where you present those patterns and stories to an audience. In the exploratory analysis you are testing out many different hypothesis, and in the explanatory analysis you pick out only the visualizations that answer the questions you are trying to answer and you present them with the audience in mind.

Explanatory data analysis goes beyond data science and statistics and combines it with the skills of artistic design and story telling.

## Part 2: Interactive visualizations with Bokeh



To really master interactive visualizations, you will need to work with JavaScript, especially [D3](https://d3js.org). Given the limited time available for this class, we can't squeeze that in. But luckily Python has some pretty good options for interactive visualizations. You can find a range of different options [here](https://mode.com/blog/python-interactive-plot-libraries/).

Today, we'll explore [`Bokeh`](https://docs.bokeh.org/en/latest/), which provides lots of nice interactive funtionalities to Python. To work with Bokeh, we first set up our system:

1. If you haven't installed it yet please do so. You can simply follow [these steps](https://docs.bokeh.org/en/latest/docs/first_steps/installation.html)

2. To include Bokeh in your notebooks you can follow the [Bokeh: Using with Jupyter](https://docs.bokeh.org/en/latest/docs/user_guide/output/jupyter.html#jupyter) guide. Come back to this one when you need it

3. We aim to give you a gentle start with Bokeh and I am going to include more example code than usual in the follwing.
   * **HINT 1**: If you're not an experienced Python user, I recommend going to the [official user's guide](https://docs.bokeh.org/en/latest/docs/user_guide.html#userguide) and working through it. Start by clicking "Introduction" in the linked page. That page has a glossary, a section on output methods, stuff on settings, and interfaces that you can scroll through. The next page *Basic Plotting* where the action is. Spend some time working through that.
   * **HINT 2**: And by "working through it", I mean copy, paste, and run the code in your own notebook. 

Ok. Let's get started. First a general announcement on the data.

> **Announcement**
> * During this entire lecture, as always, we are going to work with the SF Crime Data. 
> * We will use data for the **period 2010-2017***.


Now, to get you in the mood here's a little gif to illustrate what the goal of this exercise is:

![Movie](https://github.com/suneman/socialdata2021/blob/main/files/week8_1.gif?raw=true)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, FactorRange, Legend
from bokeh.palettes import Category20

In [2]:
# read crime data
data = pd.read_csv('/Users/sorenbendtsen/Documents/DTU Business Analytics/1st Semester/Sunessocialdata2023/files/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv')

focuscrimes = ['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT']
data = data[data.Category.isin(focuscrimes)]

# Create a subset of relevant columns
data = data[['PdId', 'Date', 'Time', 'Category', 'Descript', 'PdDistrict', 'Resolution', 'DayOfWeek', 'X', 'Y', 'location']]

# Convert Data to datetime
data["Date"] = pd.to_datetime(data["Date"])

# Create relevant datetime variables.
data["Year"] = data["Date"].dt.year
# Make a subset of data between 2010 and 2017
data = data[(data.Year >= 2010) & (data.Year <= 2017)]

data["Month"] = data["Date"].dt.month
data["MonthOfYear"] = data["Date"].dt.strftime('%b')
data["Day"] = data["Date"].dt.day
data["Hour"] = pd.to_datetime(data["Time"], format = '%H:%M').dt.hour
data["Weekday"] = pd.to_datetime(data.Date).dt.weekday
data["Week_hour"] = data.Weekday*24 + data.Hour


In [ ]:
# Data preparation for plotting with Bokeh
# We want a pivot table where each column is a crime type and each row is a hour of the day.
# To compare how easy the distribution differs between crime types, we normalize the data within each crime type.

# Group data by hour and crime type
crime_hour = data.groupby(['Category', 'Hour']).size().reset_index(name='Count')

# Normalize data within each category
crime_hour['Count'] = crime_hour.groupby('Category')['Count'].apply(lambda x: x/x.sum())

# Transpose all categories into columns 
crime_hour = crime_hour.pivot(index='Hour', columns='Category', values='Count').reset_index()

# convert to data frame
crime_hour = pd.DataFrame(crime_hour)

# Hour array with strings to use for x_range in Bokeh
hour_list = np.array(range(0,24)).astype(str)

In [18]:
# convert dataframe to Bokeh ColumnDataSource
source = ColumnDataSource(crime_hour)

In [ ]:
# clear output
from bokeh.io import curdoc
curdoc().clear()

In [39]:
# Create empty figure
p = figure(plot_width=800, plot_height=400, x_range=FactorRange(factors=hour_list), 
           x_axis_label='Hour of the day', y_axis_label='Relative frequency', title='Crimes per hour')

# Add the bars
bar = {}

for indx, i in enumerate(focuscrimes):
    bar[i] = p.vbar(x='Hour', top=i, width=0.8, source=source, color=Category20[20][indx], 
                    legend_label=i, muted_alpha=0.1, muted = True)

# Set the legend
p.legend.click_policy="mute"
p.legend.orientation = "vertical"
# place the legend outside the plot
p.add_layout(p.legend[0], 'left')

show(p)



> ***Exercise***: Recreate the results from **Week 2** as an interactive visualisation (shown in the gif). To complete the exercise, follow the steps below to create your own version of the dataviz.

### Data prep

A key step is to set up the data right. So for this one, we'll be pretty strict about the steps. The workflow is

1. Take the data for the period of 2010-2017 and group it by hour-of-the-day.
2. We would like to be able to easily compare how the distribution of crimes differ from each other, not absolute numbers, so we will work on *normalized data*:
    * To normalise data for within a crime category you simply to devide the count for each hour by the total number of this crime type. (To give a concrete example in the `ASSAULT` category, take the number of assault-counts in 1st hour you should devide by the total number of assaults, then you devide number of assaults in 2nd hour by the total number of assaults and so on)
    *  Your life will be easiest if you organize your dataframe as shown in [this helpful screenshot](https://github.com/suneman/socialdataanalysis2020/blob/master/files/week8_1_data.PNG?raw=true).

If you've followed these steps, your data should be ready! Take a moment to celebrate. We now follow the [Bokeh guide for categorical data](https://docs.bokeh.org/en/latest/docs/user_guide/basic/bars.html):



1. First, let's convert our **Pandas Dataframe** to **Bokeh ColumnDataSource**: 
  > ```python
  > source = ColumnDataSource(your_processed_dataframe)
  > ## it is a standard way to convert your df to bokeh
  > ```
2. We also need to create an empty figure (we will add our stuff here later on). Mini sub-exercise: Find the a guide how to define a figure in Bokeh online. Here is a little help:
  > ```python
  > p = figure(...., x_range = FactorRange(factors=hours), ...) 
  > #p is a standard way to call figures in Bokeh
  > #make sure to add x_range. In my case hours is a list on the form ['1', '2', '3' ... , '24']
  > #read up on the FactorRange in the guide
  > #do not forget to add other attributes to the figure, e.g. title, axis names and so on
  > ```
3. Now we are going to add the bars. In order to do so, we will use **vbar** (see the guide for help):
  > ```python
  > bar ={} # to store vbars
  > ### here we will do a for loop:
  > for indx,i in enumerate(focuscrimes):
  >     bar[i] = p.vbar(x='name_of_the_column_that_contain_hours',  top=i, source= src, 
  >                     ### we will create a vbar for each focuscrime
  >                     legend_label=i,  muted_alpha=..., muted = ....) 
  > #i stands for a column that we use, top=y; we are specifying that our numbers comes from column i
  > #read up what legend_label, muted and muted_alpha do... you can add more attributes (you HAVE TO)
  > ```
4. The last thing to do is to make your legend interactive and display the figure:
  > ```python
  > p.legend.click_policy="mute" ### assigns the click policy (you can try to use ''hide'
  > show(p) #displays your plot
  > ```
5. You will notice that the legend appears in the middle of the figure (and it occludes some of the data). In order to fix this look into [this guide](https://stackoverflow.com/questions/26254619/position-of-the-legend-in-a-bokeh-plot) as a start. Below are some code snippets that you can use to deal with this problem (but read the guide first):
  > ```python
  > items = [] ### for the custom legend // you need to figure out where to add it
  > items.append((i, [bar[i]])) ### figure where to add it
  > legend = Legend(items=..., location=.....) ## figure where to add it
  > p.add_layout(...., ...) ## figure where to add it
  > ### if you read the guide, it will make sense
  > ```

Now you should be able to recreate this amazing visualisation.


**EXTRA feature**: If you're interested in detailed instructions for more Bokeh visualizations for your final project, you can find more inspiration **[here](https://github.com/suneman/socialdata2021/blob/main/lectures/Week8_extra_bokeh.ipynb)**.

## Part 3: Narrative Dataviz

Let's finish up with some reading

*Reading*: [Narrative Visualization: Telling Stories with Data](http://vis.stanford.edu/files/2010-Narrative-InfoVis.pdf) by Edward Segel and Jeffrey Heer. We'll read section 1-3 today. (And the rest a bit later).

When you get to section 3 it's fun to open up the examples mentioned by the authors in a browser and explore them as you read the text. 

> *Exercise*: Answer a couple of questions about the paper.
> 
> * What is the *Oxford English Dictionary's* defintion of a narrative?
> * What is your favorite visualization among the examples in section 3? Explain why in a few words.

**Oxford English dixtionary's definition of a narratice**
The Oxford English Dictionary defines narrative as “an account of a series of events, facts, etc., given in order and with the establishing of connections between them.” Central to this definition is the notion of a chain of causally related events. Stories of this form often have a beginning, middle, and end [3, 24]: an introduction to the situation, a series of events often involving tension or conflict, and a resolution.

**Favorite visualization**
My favorite visualization is Figure 4 on Human Development Trends. This is because the visualization is from 2005! The Gapminder platform definitely looks outdated, but based on technology from 18 years ago, I was very impressed on the great story telling they were able to do. This shows the power of using visual highlighting, guiding the audience, consistent use of colors etc. 